In [1]:
%matplotlib widget
import sys

sys.path.append("..")

import matplotlib.pyplot as plt
import numpy as np
from matplotlib.widgets import Slider
from mpl_toolkits.mplot3d import Axes3D

from app.face_mesh_landmark_parts_index import parts_index

In [2]:
direction = np.load("face_direction.npy")
direction = direction[:, 0, :, :]

direction.shape

(518, 468, 3)

In [3]:
def plot_landmarks(landmarks):
    landmarks = np.array([l - l.mean(axis=0) for l in landmarks])
    fig = plt.figure(figsize=(6, 4))
    gs = fig.add_gridspec(12, 1)
    ax = fig.add_subplot(gs[:-2, :], projection="3d")
    ax_slider = fig.add_subplot(gs[-1, :])
    slider = Slider(ax_slider, "index", 0, len(landmarks), valinit=0, valstep=1)

    scatters = [ax.scatter3D(*landmarks[idx, :, :].T) for idx in range(len(landmarks))]
    for s in scatters:
        s.set_visible(False)

    def update(v):
        for s in [s for s in scatters if s.get_visible()]:
            s.set_visible(False)
        idx = int(slider.val)
        scatters[idx].set_visible(True)

    slider.on_changed(update)
    update(None)

    ax.set(
        xlim=(-0.2, 0.2),
        ylim=(-0.3, 0.3),
        zlim=(-0.2, 0.2),
        xlabel="x",
        ylabel="y",
        zlabel="z",
    )
    ax.view_init(elev=95, azim=90)
    fig.tight_layout()


plot_landmarks(direction)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
def plot_bias(landmarks):
    landmarks = np.array([l - l.mean(axis=0) for l in landmarks])
    x_bias = [
        np.count_nonzero(l[:, 0] < 0) / np.count_nonzero(l[:, 0] > 0) for l in landmarks
    ]
    y_bias = [
        np.count_nonzero(l[:, 1] < 0) / np.count_nonzero(l[:, 1] > 0) for l in landmarks
    ]

    fig, ax1 = plt.subplots(figsize=(5, 2))
    ax2 = ax1.twinx()
    ax1.plot(x_bias, color="red")
    ax1.set_ylabel("x bias", color="red")
    ax1.tick_params(axis="y", colors="red")
    ax1.spines[ax1.yaxis.get_ticks_position()].set_color("red")

    ax2.plot(y_bias, color="blue")
    ax2.set_ylabel("y bias", color="blue")
    ax2.tick_params(axis="y", colors="blue")
    ax2.spines[ax2.yaxis.get_ticks_position()].set_color("blue")
    fig.tight_layout()

plot_bias(direction)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
def plot_eye_lip_angle(landmarks):
    landmarks = np.array([l - l.mean(axis=0) for l in landmarks])
    left_eye = landmarks[:, parts_index["left_eye"], :].mean(axis=1)
    right_eye = landmarks[:, parts_index["right_eye"], :].mean(axis=1)
    xy_eye_slope = (right_eye[:, 1] - left_eye[:, 1]) / (
        right_eye[:, 0] - left_eye[:, 0]
    )
    xz_eye_slope = (right_eye[:, 2] - left_eye[:, 2]) / (
        right_eye[:, 0] - left_eye[:, 0]
    )

    fig, ax1 = plt.subplots(figsize=(5, 2))
    ax2 = ax1.twinx()
    ax1.plot(xy_eye_slope, color="red")
    ax1.set_ylabel("xy eye angle", color="red")
    ax1.tick_params(axis="y", colors="red")
    ax1.spines[ax1.yaxis.get_ticks_position()].set_color("red")

    ax2.plot(xz_eye_slope, color="blue")
    ax2.set_ylabel("xz eye angle", color="blue")
    ax2.tick_params(axis="y", colors="blue")
    ax2.spines[ax2.yaxis.get_ticks_position()].set_color("blue")
    fig.tight_layout()

    eye_center = (left_eye + right_eye) / 2
    upper_lip = landmarks[:, parts_index["upper_lip"], :].mean(axis=1)
    lower_lip = landmarks[:, parts_index["lower_lip"], :].mean(axis=1)
    lip_center = (upper_lip + lower_lip) / 2

    yz_eye_lip_slope = (eye_center[:, 2] - lip_center[:, 2]) / (eye_center[:, 1] - lip_center[:, 1])
    fig, ax = plt.subplots(figsize=(5, 2))
    ax.plot(yz_eye_lip_slope - yz_eye_lip_slope[0])


plot_eye_lip_angle(direction)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …